# WORLD BANK DATA ANALYSIS

*For this project I am using the pandas and json libraries and the json_normalize function. In the code below, I import said libraries and function and also load the json file both as a string and as a dataframe* 

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize 

### load file into json object for later use with json.normalize in part 2 and 3
with open('data_wrangling_json/data/world_bank_projects.json') as json_file:
    json_data=json.load(json_file)
    
### load file into dataframe object for use in part 1 and 3
df=pd.DataFrame(json_data)


## PART 1

*To find the 10 countries with the most projects we can simply groupby countryname and count the number of rows in each group (country)*

In [2]:
df_srt1=df.groupby('countryname').count() #grouping by countryname and counting number of rows  

df_srt1=df_srt1.sort_values(ascending=False,by='id') # putting in descending order

print(df_srt1['id'].head(10))

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Nepal                              12
Kingdom of Morocco                 12
Republic of Mozambique             11
Africa                             11
Name: id, dtype: int64



*As you can see on the last row, Africa is counted as a country. The next code block drops that row so that the output makes more sense.*


In [3]:
df_srt1.drop('Africa',inplace=True) #dropping rows with Africa

df_srt1['id'].head(10)

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Nepal                              12
Kingdom of Morocco                 12
Republic of Mozambique             11
Burkina Faso                        9
Name: id, dtype: int64

##                                                                  PART 2

*To list the top 10 themes, I create a dataframe called 'theme_codes' that returns a dataframe with only the theme names and theme codes for each project. In the dataframe, I also include the 'id' column so that values in the theme code and name columns are preserved.*

*I can then groupby 'theme code' and 'theme name', creating a two level index dataframe, aggregate by count and sort the values by id as before in part 1.*

**Note:** *This process does not count all the rows with the same themecode. This is because for rows of a given themecode, the groupby method seperates rows with unfilled theme names from rows with filled theme names. This does not seem to effect the order of the themes with the most projects, as can be verified uncommenting the second code block of this part*

In [4]:
theme_codes=json_normalize(json_data,'mjtheme_namecode','id')# creating dataframe from json string with theme_code, theme name, and id

theme_codes.columns=['theme code','theme name','id']

theme_codes_srt=theme_codes.groupby(['theme code','theme name']).count().sort_values(by='id',ascending=False)# grouping, counting and sorting

theme_codes_srt.columns=['# of projects']

print(theme_codes_srt.head(10))

                                                         # of projects
theme code theme name                                                 
11         Environment and natural resources management            223
10         Rural development                                       202
8          Human development                                       197
2          Public sector governance                                184
6          Social protection and risk management                   158
4          Financial and private sector development                130
7          Social dev/gender/inclusion                             119
5          Trade and integration                                    72
9          Urban development                                        47
1          Economic management                                      33


*Uncomment the code below to verify that grouping without the unfilled theme names still produces the same order*

In [5]:
#theme_codes_srt_with_unfilled=theme_codes.groupby(['theme code']).count().sort_values(by='id',ascending=False)
#print(theme_codes_srt_with_unfilled.id.head(10))

theme code
11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: id, dtype: int64


##                                                                  PART 3

*To fill in the missing theme names, I drop the rows with blank 'theme name' indicies and use the resulting dataframe's Index attribute to create a look up dictionary with keys of themecodes and values of their corresponding theme names. I then create a function to replace each cell of mjtheme with the name corresponding to its themecode *

In [6]:
theme_codes_srt=theme_codes_srt.drop('',level=1)# dropping blank theme names from theme_codes_srt

cds2nms= dict(list(theme_codes_srt.index.values))# creating dict of themecodes to themenames 

def fill_missing_names(list_dict):
### Takes a cell of mjtheme_namecode which contains a list of dicts of themecode: themename key:value pairs...
  # that are associated with each project and returns a list of theme names that can be set as the corresponding...   
  # mjtheme cell's value 
###
    
    
    list_names=[] #empty list to be appended to and returned
    
    for name_code_dict in list_dict:
    ## This for loop iterates over each dict in the list of dicts that makes up each cell of mjtheme_namecode...
     # and appends to list_names the theme name corresponding to the themecode key of each dict using the cds2nms...
     # dict created above
    ##
    
        list_names.append(cds2nms[name_code_dict['code']]) #appending theme name to list
        
    return list_names


df.mjtheme=df.mjtheme_namecode.apply(fill_missing_names)# applying function

print(df.mjtheme)

0      [Human development, Environment and natural re...
1      [Economic management, Social protection and ri...
2      [Trade and integration, Public sector governan...
3      [Social dev/gender/inclusion, Social dev/gende...
4      [Trade and integration, Financial and private ...
5      [Social protection and risk management, Social...
6      [Public sector governance, Financial and priva...
7      [Environment and natural resources management,...
8       [Rural development, Social dev/gender/inclusion]
9      [Public sector governance, Public sector gover...
10         [Rural development, Public sector governance]
11     [Rural development, Social protection and risk...
12            [Financial and private sector development]
13     [Trade and integration, Environment and natura...
14     [Social protection and risk management, Social...
15     [Rural development, Environment and natural re...
16         [Rural development, Public sector governance]
17      [Human development, Soc